<a href="https://colab.research.google.com/github/AnishSharma1/Science-Fair-Project/blob/main/Untitled34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Integrated Multi-Omic Molecular Mimicry Analysis Pipeline
===========================================================
Combines:
1. TCR binding energetics (from enhanced analyzer)
2. Peptide sequence/structure similarity (from PDB analyzer)
3. Bulk RNA-seq data (gene expression)
4. Clinical metadata (patient outcomes)

Creates unified mimicry score and multi-omic integration
"""

# Installation
print("Installing packages...")
!pip install biopython pandas matplotlib seaborn scipy scikit-learn umap-learn adjustText -q
print("✓ Complete!")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.cluster import hierarchy
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from google.colab import files
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("INTEGRATED MULTI-OMIC MOLECULAR MIMICRY PIPELINE")
print("="*70)

# ============================================================================
# PART 1: LOAD AND INTEGRATE STRUCTURAL DATA
# ============================================================================
print("\n" + "="*70)
print("PART 1: STRUCTURAL DATA INTEGRATION")
print("="*70)

print("\n📁 Upload TCR-matched predictions CSV:")
tcr_file = files.upload()
tcr_filename = list(tcr_file.keys())[0]
tcr_df = pd.read_csv(tcr_filename)
print(f"✓ Loaded {len(tcr_df)} TCR-matched predictions")

print("\n📁 Upload PDB similarity/cross-reactivity CSV:")
pdb_file = files.upload()
pdb_filename = list(pdb_file.keys())[0]
pdb_df = pd.read_csv(pdb_filename)
print(f"✓ Loaded {len(pdb_df)} PDB similarity scores")

# ============================================================================
# PART 2: CREATE UNIFIED MIMICRY SCORE
# ============================================================================
print("\n" + "="*70)
print("PART 2: UNIFIED MOLECULAR MIMICRY SCORE CALCULATION")
print("="*70)

def create_unified_mimicry_score(tcr_df, pdb_df):
    """
    Combine TCR binding energetics with peptide similarity

    TCR component: How well TCR recognizes both peptides
    Peptide component: How similar the peptides are (sequence + structure)

    Final score = weighted combination reflecting TRUE mimicry risk
    """

    # Merge datasets on EBV and Myelin file identifiers
    print("\n🔗 Merging TCR and PDB datasets...")

    # Normalize column names
    tcr_df_norm = tcr_df.copy()
    pdb_df_norm = pdb_df.copy()

    # Try different merge strategies
    # Strategy 1: Direct merge on filenames
    if 'EBV_File' in tcr_df_norm.columns and 'EBV' in pdb_df_norm.columns:
        # Clean filenames for matching
        tcr_df_norm['EBV_clean'] = tcr_df_norm['EBV_File'].str.replace('.pdb', '').str.replace(' (1)', '').str.replace(' (2)', '')
        tcr_df_norm['Myelin_clean'] = tcr_df_norm['Myelin_File'].str.replace('.pdb', '').str.replace(' (1)', '').str.replace(' (2)', '')

        pdb_df_norm['EBV_clean'] = pdb_df_norm['EBV'].str.replace('.pdb', '').str.replace(' (1)', '').str.replace(' (2)', '')
        pdb_df_norm['Myelin_clean'] = pdb_df_norm['Myelin'].str.replace('.pdb', '').str.replace(' (1)', '').str.replace(' (2)', '')

        # Merge
        merged_df = pd.merge(
            tcr_df_norm,
            pdb_df_norm,
            left_on=['EBV_clean', 'Myelin_clean'],
            right_on=['EBV_clean', 'Myelin_clean'],
            how='inner',
            suffixes=('_tcr', '_pdb')
        )

        print(f"✓ Merged {len(merged_df)} matching pairs")
    else:
        print("⚠ Column name mismatch - using all data separately")
        merged_df = None

    if merged_df is not None and len(merged_df) > 0:
        # Calculate unified mimicry score
        print("\n🧮 Calculating Unified Molecular Mimicry Score...")

        # Component 1: TCR Recognition Similarity (0-100)
        # High score = TCR binds both peptides similarly well
        if 'TCR_Matched_Score' in merged_df.columns:
            tcr_component = merged_df['TCR_Matched_Score']
        elif 'Total_TCR_Energy' in merged_df.columns:
            # Normalize energy to 0-100 scale
            tcr_component = 100 - MinMaxScaler(feature_range=(0, 100)).fit_transform(
                merged_df[['Total_TCR_Energy']].abs()
            ).flatten()
        else:
            tcr_component = 50  # Default if no TCR data

        # Component 2: Peptide Similarity (0-100)
        # High score = peptides are structurally/sequentially similar
        if 'Cross-Reactivity Score' in merged_df.columns:
            peptide_component = merged_df['Cross-Reactivity Score']
        elif 'Risk_Mean' in merged_df.columns:
            peptide_component = merged_df['Risk_Mean']
        else:
            peptide_component = 50  # Default

        # Component 3: Structural Quality Weight
        # Weight by AlphaFold confidence - only trust high-quality structures
        if 'Myelin_pLDDT' in merged_df.columns and 'EBV_pLDDT' in merged_df.columns:
            avg_plddt = (merged_df['Myelin_pLDDT'] + merged_df['EBV_pLDDT']) / 2
            quality_weight = avg_plddt / 100
        else:
            quality_weight = 1.0

        # Component 4: Binding Energy Similarity
        if 'EBV_Binding_Energy' in merged_df.columns and 'Myelin_Binding_Energy' in merged_df.columns:
            energy_diff = abs(merged_df['EBV_Binding_Energy'] - merged_df['Myelin_Binding_Energy'])
            max_energy = merged_df[['EBV_Binding_Energy', 'Myelin_Binding_Energy']].abs().max(axis=1)
            energy_similarity = 100 * (1 - energy_diff / max_energy)
        else:
            energy_similarity = 50

        # UNIFIED MIMICRY SCORE FORMULA
        # High score = Strong molecular mimicry (dangerous!)
        merged_df['Unified_Mimicry_Score'] = (
            tcr_component * 0.35 +           # TCR recognition
            peptide_component * 0.35 +       # Peptide similarity
            energy_similarity * 0.30         # Energy similarity
        ) * quality_weight

        # Component scores for interpretation
        merged_df['TCR_Component'] = tcr_component
        merged_df['Peptide_Component'] = peptide_component
        merged_df['Energy_Component'] = energy_similarity
        merged_df['Quality_Weight'] = quality_weight

        # Classification
        merged_df['Mimicry_Class'] = pd.cut(
            merged_df['Unified_Mimicry_Score'],
            bins=[0, 30, 50, 70, 100],
            labels=['Low', 'Moderate', 'High', 'Critical']
        )

        print(f"\n📊 Mimicry Score Distribution:")
        print(f"   Mean: {merged_df['Unified_Mimicry_Score'].mean():.1f}")
        print(f"   Median: {merged_df['Unified_Mimicry_Score'].median():.1f}")
        print(f"   Range: {merged_df['Unified_Mimicry_Score'].min():.1f} - {merged_df['Unified_Mimicry_Score'].max():.1f}")
        print(f"\n   Critical (>70): {(merged_df['Unified_Mimicry_Score'] > 70).sum()}")
        print(f"   High (50-70): {((merged_df['Unified_Mimicry_Score'] > 50) & (merged_df['Unified_Mimicry_Score'] <= 70)).sum()}")
        print(f"   Moderate (30-50): {((merged_df['Unified_Mimicry_Score'] > 30) & (merged_df['Unified_Mimicry_Score'] <= 50)).sum()}")
        print(f"   Low (<30): {(merged_df['Unified_Mimicry_Score'] <= 30).sum()}")

        return merged_df
    else:
        return None

integrated_df = create_unified_mimicry_score(tcr_df, pdb_df)

if integrated_df is not None:
    # Export integrated scores
    integrated_df.to_csv('unified_mimicry_scores.csv', index=False)
    print("\n✓ unified_mimicry_scores.csv")
    files.download('unified_mimicry_scores.csv')

# ============================================================================
# PART 3: BULK RNA-SEQ INTEGRATION
# ============================================================================
print("\n" + "="*70)
print("PART 3: BULK RNA-SEQ DATA INTEGRATION")
print("="*70)

print("\n📁 Upload bulk RNA-seq count matrix (CSV):")
print("   Format: Rows=genes, Columns=samples")
print("   First column should be gene names/IDs")
rnaseq_upload = input("Do you have RNA-seq data to upload? (yes/no): ")

rnaseq_df = None
if rnaseq_upload.lower() == 'yes':
    rnaseq_file = files.upload()
    rnaseq_filename = list(rnaseq_file.keys())[0]
    rnaseq_df = pd.read_csv(rnaseq_filename, index_col=0)
    print(f"✓ Loaded expression data: {rnaseq_df.shape[0]} genes × {rnaseq_df.shape[1]} samples")

    # Basic QC and normalization
    print("\n🔬 RNA-seq preprocessing:")

    # Remove low-expression genes
    min_expression = 10
    expressed_genes = (rnaseq_df.sum(axis=1) > min_expression)
    rnaseq_df = rnaseq_df[expressed_genes]
    print(f"   • Filtered to {len(rnaseq_df)} expressed genes (sum > {min_expression})")

    # Log2(CPM + 1) normalization
    library_sizes = rnaseq_df.sum(axis=0)
    cpm = rnaseq_df.div(library_sizes, axis=1) * 1e6
    rnaseq_norm = np.log2(cpm + 1)
    print(f"   • Normalized: log2(CPM + 1)")

    # Identify immune/myelin-related genes
    immune_genes = ['CD4', 'CD8A', 'CD8B', 'IFNG', 'TNF', 'IL2', 'IL10', 'IL17A',
                    'FOXP3', 'GZMB', 'PRF1', 'CXCR3', 'CCR5', 'TBX21', 'GATA3']
    myelin_genes = ['MBP', 'PLP1', 'MOG', 'MAG', 'CNP', 'MOBP']
    ebv_response_genes = ['EBNA1', 'LMP1', 'LMP2A', 'IRF4', 'EBI3']

    genes_of_interest = immune_genes + myelin_genes + ebv_response_genes
    present_genes = [g for g in genes_of_interest if g in rnaseq_norm.index]

    print(f"\n   📋 Key genes detected: {len(present_genes)}/{len(genes_of_interest)}")
    print(f"      {', '.join(present_genes[:10])}...")

    # Calculate gene signatures
    if len(present_genes) > 0:
        gene_signatures = pd.DataFrame(index=rnaseq_norm.columns)

        immune_present = [g for g in immune_genes if g in rnaseq_norm.index]
        if immune_present:
            gene_signatures['Immune_Score'] = rnaseq_norm.loc[immune_present].mean()

        myelin_present = [g for g in myelin_genes if g in rnaseq_norm.index]
        if myelin_present:
            gene_signatures['Myelin_Score'] = rnaseq_norm.loc[myelin_present].mean()

        ebv_present = [g for g in ebv_response_genes if g in rnaseq_norm.index]
        if ebv_present:
            gene_signatures['EBV_Response_Score'] = rnaseq_norm.loc[ebv_present].mean()

        print("\n   ✓ Calculated gene signatures")

# ============================================================================
# PART 4: CLINICAL METADATA INTEGRATION
# ============================================================================
print("\n" + "="*70)
print("PART 4: CLINICAL METADATA INTEGRATION")
print("="*70)

print("\n📁 Upload clinical metadata (CSV):")
print("   Should include: Sample_ID, Disease_Status, Age, Sex, etc.")
clinical_upload = input("Do you have clinical data to upload? (yes/no): ")

clinical_df = None
if clinical_upload.lower() == 'yes':
    clinical_file = files.upload()
    clinical_filename = list(clinical_file.keys())[0]
    clinical_df = pd.read_csv(clinical_filename)
    print(f"✓ Loaded clinical data: {len(clinical_df)} samples")
    print(f"   Columns: {', '.join(clinical_df.columns.tolist())}")

    # Display basic statistics
    if 'Disease_Status' in clinical_df.columns:
        print(f"\n   Disease distribution:")
        print(clinical_df['Disease_Status'].value_counts().to_string())

# ============================================================================
# PART 5: MULTI-OMIC INTEGRATION
# ============================================================================
print("\n" + "="*70)
print("PART 5: MULTI-OMIC INTEGRATION & CORRELATION ANALYSIS")
print("="*70)

if rnaseq_df is not None and clinical_df is not None and integrated_df is not None:
    print("\n🔗 Integrating structural, transcriptomic, and clinical data...")

    # Create integrated analysis
    # Link peptide pairs to patient samples based on matching criteria

    # For demonstration, we'll create correlation analyses
    multi_omic_results = []

    # Check if we can link samples
    if 'Sample_ID' in clinical_df.columns and 'Sample_ID' in gene_signatures.index:
        # Merge gene signatures with clinical data
        integrated_clinical = pd.merge(
            clinical_df,
            gene_signatures,
            left_on='Sample_ID',
            right_index=True,
            how='inner'
        )

        print(f"✓ Integrated {len(integrated_clinical)} samples with both clinical and expression data")

        # Analyze relationships
        if 'Disease_Status' in integrated_clinical.columns:
            print("\n📊 Gene Signature by Disease Status:")
            for col in gene_signatures.columns:
                if col in integrated_clinical.columns:
                    print(f"\n   {col}:")
                    grouped = integrated_clinical.groupby('Disease_Status')[col].agg(['mean', 'std', 'count'])
                    print(grouped.to_string())

                    # Statistical test
                    groups = integrated_clinical.groupby('Disease_Status')[col].apply(list)
                    if len(groups) == 2:
                        stat, pval = stats.ttest_ind(groups.iloc[0], groups.iloc[1])
                        print(f"   t-test p-value: {pval:.4e}")

# ============================================================================
# PART 6: COMPREHENSIVE VISUALIZATIONS
# ============================================================================
print("\n" + "="*70)
print("PART 6: COMPREHENSIVE MULTI-OMIC VISUALIZATIONS")
print("="*70)

if integrated_df is not None:
    fig = plt.figure(figsize=(20, 16))
    gs = fig.add_gridspec(4, 3, hspace=0.35, wspace=0.3)

    # 1. Unified Mimicry Score Distribution
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.hist(integrated_df['Unified_Mimicry_Score'], bins=30,
            color='purple', edgecolor='black', alpha=0.7)
    ax1.axvline(70, color='red', linestyle='--', linewidth=2, label='Critical threshold')
    ax1.axvline(50, color='orange', linestyle='--', linewidth=2, label='High threshold')
    ax1.set_xlabel('Unified Mimicry Score', fontsize=11)
    ax1.set_ylabel('Frequency', fontsize=11)
    ax1.set_title('Unified Mimicry Score Distribution', fontsize=13, fontweight='bold')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # 2. Component Contribution
    ax2 = fig.add_subplot(gs[0, 1])
    components = ['TCR_Component', 'Peptide_Component', 'Energy_Component']
    component_means = [integrated_df[c].mean() for c in components]
    colors_comp = ['orchid', 'lightcoral', 'lightblue']
    bars = ax2.bar(range(3), component_means, color=colors_comp, edgecolor='black', linewidth=1.5)
    ax2.set_xticks(range(3))
    ax2.set_xticklabels(['TCR\nRecognition', 'Peptide\nSimilarity', 'Energy\nSimilarity'], fontsize=10)
    ax2.set_ylabel('Mean Score', fontsize=11)
    ax2.set_title('Mimicry Score Components', fontsize=13, fontweight='bold')
    ax2.set_ylim([0, 100])
    ax2.grid(axis='y', alpha=0.3)

    # Add values on bars
    for i, (bar, val) in enumerate(zip(bars, component_means)):
        ax2.text(bar.get_x() + bar.get_width()/2, val + 2, f'{val:.1f}',
                ha='center', va='bottom', fontsize=10, fontweight='bold')

    # 3. TCR vs Peptide Components
    ax3 = fig.add_subplot(gs[0, 2])
    scatter = ax3.scatter(integrated_df['TCR_Component'],
                         integrated_df['Peptide_Component'],
                         c=integrated_df['Unified_Mimicry_Score'],
                         s=80, cmap='RdYlGn_r', alpha=0.6,
                         edgecolors='black', linewidth=0.5)
    ax3.plot([0, 100], [0, 100], 'k--', alpha=0.3, linewidth=1)
    ax3.set_xlabel('TCR Recognition Score', fontsize=11)
    ax3.set_ylabel('Peptide Similarity Score', fontsize=11)
    ax3.set_title('Component Correlation', fontsize=13, fontweight='bold')
    plt.colorbar(scatter, ax=ax3, label='Unified Score')
    ax3.grid(alpha=0.3)

    # 4. Top 10 mimicry pairs
    ax4 = fig.add_subplot(gs[1, :])
    top10 = integrated_df.nlargest(10, 'Unified_Mimicry_Score')
    y_pos = np.arange(len(top10))

    # Create color map based on classification
    colors_risk = {'Critical': 'red', 'High': 'orange', 'Moderate': 'yellow', 'Low': 'green'}
    bar_colors = [colors_risk.get(c, 'gray') for c in top10['Mimicry_Class']]

    bars = ax4.barh(y_pos, top10['Unified_Mimicry_Score'],
                    color=bar_colors, edgecolor='black', linewidth=1.5, alpha=0.8)
    ax4.set_yticks(y_pos)

    # Create labels
    labels = []
    for idx, row in top10.iterrows():
        ebv_short = row.get('EBV_clean', row.get('EBV_File', 'Unknown'))[:25]
        myelin_short = row.get('Myelin_clean', row.get('Myelin_File', 'Unknown'))[:25]
        labels.append(f"{ebv_short}\n↔ {myelin_short}")

    ax4.set_yticklabels(labels, fontsize=8)
    ax4.set_xlabel('Unified Mimicry Score', fontsize=11)
    ax4.set_title('Top 10 Molecular Mimicry Candidates', fontsize=13, fontweight='bold')
    ax4.invert_yaxis()
    ax4.grid(axis='x', alpha=0.3)
    ax4.axvline(70, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax4.axvline(50, color='orange', linestyle='--', linewidth=1, alpha=0.5)

    # Add score text
    for i, (bar, score) in enumerate(zip(bars, top10['Unified_Mimicry_Score'])):
        ax4.text(score + 1, bar.get_y() + bar.get_height()/2,
                f'{score:.1f}', va='center', fontsize=9, fontweight='bold')

    # 5. Quality weight impact
    ax5 = fig.add_subplot(gs[2, 0])
    ax5.scatter(integrated_df['Quality_Weight'],
               integrated_df['Unified_Mimicry_Score'],
               c=integrated_df['Peptide_Component'],
               s=60, cmap='viridis', alpha=0.6, edgecolors='black', linewidth=0.5)
    ax5.set_xlabel('Quality Weight (Avg pLDDT)', fontsize=11)
    ax5.set_ylabel('Unified Mimicry Score', fontsize=11)
    ax5.set_title('Structure Quality Impact', fontsize=13, fontweight='bold')
    ax5.grid(alpha=0.3)

    # 6. Mimicry class distribution
    ax6 = fig.add_subplot(gs[2, 1])
    class_counts = integrated_df['Mimicry_Class'].value_counts()
    colors_pie = ['green', 'yellow', 'orange', 'red']
    wedges, texts, autotexts = ax6.pie(class_counts.values,
                                        labels=class_counts.index,
                                        colors=colors_pie,
                                        autopct='%1.1f%%',
                                        startangle=90,
                                        explode=[0.05]*len(class_counts))
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
    ax6.set_title('Mimicry Risk Classification', fontsize=13, fontweight='bold')

    # 7. Component correlation heatmap
    ax7 = fig.add_subplot(gs[2, 2])
    corr_data = integrated_df[['TCR_Component', 'Peptide_Component',
                                'Energy_Component', 'Unified_Mimicry_Score']].corr()
    sns.heatmap(corr_data, annot=True, fmt='.3f', cmap='coolwarm',
               center=0, vmin=-1, vmax=1, ax=ax7,
               square=True, linewidths=1, cbar_kws={'label': 'Correlation'})
    ax7.set_title('Component Correlations', fontsize=13, fontweight='bold')

    # 8. RNA-seq integration (if available)
    if rnaseq_df is not None and 'gene_signatures' in locals():
        ax8 = fig.add_subplot(gs[3, 0])

        if 'Immune_Score' in gene_signatures.columns:
            ax8.hist(gene_signatures['Immune_Score'], bins=20,
                    color='steelblue', edgecolor='black', alpha=0.7)
            ax8.set_xlabel('Immune Signature Score', fontsize=11)
            ax8.set_ylabel('Frequency', fontsize=11)
            ax8.set_title('Immune Gene Expression', fontsize=13, fontweight='bold')
            ax8.grid(alpha=0.3)

    # 9. Clinical correlation (if available)
    if clinical_df is not None and 'integrated_clinical' in locals():
        ax9 = fig.add_subplot(gs[3, 1])

        if 'Disease_Status' in integrated_clinical.columns and 'Immune_Score' in integrated_clinical.columns:
            disease_groups = integrated_clinical.groupby('Disease_Status')['Immune_Score'].apply(list)

            bp = ax9.boxplot(disease_groups.values, labels=disease_groups.index,
                           patch_artist=True)
            for patch in bp['boxes']:
                patch.set_facecolor('lightblue')
            ax9.set_ylabel('Immune Score', fontsize=11)
            ax9.set_title('Immune Score by Disease', fontsize=13, fontweight='bold')
            ax9.grid(axis='y', alpha=0.3)

    # 10. Summary statistics table
    ax10 = fig.add_subplot(gs[3, 2])
    ax10.axis('off')

    summary_data = [
        ['Metric', 'Value'],
        ['Total Pairs Analyzed', f"{len(integrated_df)}"],
        ['Mean Mimicry Score', f"{integrated_df['Unified_Mimicry_Score'].mean():.1f}"],
        ['Critical Risk Pairs', f"{(integrated_df['Unified_Mimicry_Score'] > 70).sum()}"],
        ['High Risk Pairs', f"{((integrated_df['Unified_Mimicry_Score'] > 50) & (integrated_df['Unified_Mimicry_Score'] <= 70)).sum()}"],
        ['Mean TCR Component', f"{integrated_df['TCR_Component'].mean():.1f}"],
        ['Mean Peptide Component', f"{integrated_df['Peptide_Component'].mean():.1f}"],
        ['Mean Quality Weight', f"{integrated_df['Quality_Weight'].mean():.3f}"],
    ]

    table = ax10.table(cellText=summary_data, cellLoc='left', loc='center',
                      colWidths=[0.65, 0.35])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2.5)

    for i in range(len(summary_data)):
        if i == 0:
            table[(i, 0)].set_facecolor('#40466e')
            table[(i, 1)].set_facecolor('#40466e')
            table[(i, 0)].set_text_props(weight='bold', color='white')
            table[(i, 1)].set_text_props(weight='bold', color='white')
        else:
            table[(i, 0)].set_facecolor('#f0f0f0' if i % 2 == 0 else 'white')
            table[(i, 1)].set_facecolor('#f0f0f0' if i % 2 == 0 else 'white')

    ax10.set_title('Summary Statistics', fontsize=13, fontweight='bold', pad=20)

    plt.suptitle('Integrated Multi-Omic Molecular Mimicry Analysis',
                fontsize=18, fontweight='bold', y=0.998)
    plt.show()

# ============================================================================
# PART 7: DIMENSIONAL REDUCTION FOR PATTERN DISCOVERY
# ============================================================================
if integrated_df is not None and len(integrated_df) > 10:
    print("\n" + "="*70)
    print("PART 7: DIMENSIONAL REDUCTION & PATTERN DISCOVERY")
    print("="*70)

    # Prepare feature matrix
    feature_cols = ['TCR_Component', 'Peptide_Component', 'Energy_Component']

    # Add additional features if available
    if 'EBV_Contacts' in integrated_df.columns:
        feature_cols.extend(['EBV_Contacts', 'Myelin_Contacts'])
    if 'EBV_HBonds' in integrated_df.columns:
        feature_cols.extend(['EBV_HBonds', 'Myelin_HBonds'])

    X = integrated_df[feature_cols].fillna(0).values

    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled)

    # t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    X_tsne = tsne.fit_transform(X_scaled)

    # UMAP
    reducer = umap.UMAP(random_state=42)
    X_umap = reducer.fit_transform(X_scaled)

    # Visualization
    fig2, axes = plt.subplots(1, 3, figsize=(18, 6))

    # PCA
    scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1],
                              c=integrated_df['Unified_Mimicry_Score'],
                              cmap='RdYlGn_r', s=80, alpha=0.6,
                              edgecolors='black', linewidth=0.5)
    axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)', fontsize=11)
    axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)', fontsize=11)
    axes[0].set_title('PCA Projection', fontsize=13, fontweight='bold')
    plt.colorbar(scatter1, ax=axes[0], label='Mimicry Score')
    axes[0].grid(alpha=0.3)

    # t-SNE
    scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1],
                              c=integrated_df['Unified_Mimicry_Score'],
                              cmap='RdYlGn_r', s=80, alpha=0.6,
                              edgecolors='black', linewidth=0.5)
    axes[1].set_xlabel('t-SNE 1', fontsize=11)
    axes[1].set_ylabel('t-SNE 2', fontsize=11)
    axes[1].set_title('t-SNE Projection', fontsize=13, fontweight='bold')
    plt.colorbar(scatter2, ax=axes[1], label='Mimicry Score')
    axes[1].grid(alpha=0.3)

    # UMAP
    scatter3 = axes[2].scatter(X_umap[:, 0], X_umap[:, 1],
                              c=integrated_df['Unified_Mimicry_Score'],
                              cmap='RdYlGn_r', s=80, alpha=0.6,
                              edgecolors='black', linewidth=0.5)
    axes[2].set_xlabel('UMAP 1', fontsize=11)
    axes[2].set_ylabel('UMAP 2', fontsize=11)
    axes[2].set_title('UMAP Projection', fontsize=13, fontweight='bold')
    plt.colorbar(scatter3, ax=axes[2], label='Mimicry Score')
    axes[2].grid(alpha=0.3)

    plt.suptitle('Dimensional Reduction Analysis', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    print(f"\n✓ PCA variance explained: {pca.explained_variance_ratio_[0]*100:.1f}% + {pca.explained_variance_ratio_[1]*100:.1f}% = {pca.explained_variance_ratio_[:2].sum()*100:.1f}%")

# ============================================================================
# PART 8: GENERATE COMPREHENSIVE REPORTS
# ============================================================================
print("\n" + "="*70)
print("PART 8: GENERATING COMPREHENSIVE REPORTS")
print("="*70)

if integrated_df is not None:
    # Top candidates report
    top_candidates = integrated_df.nlargest(20, 'Unified_Mimicry_Score')[[
        'EBV_File', 'Myelin_File', 'Unified_Mimicry_Score',
        'TCR_Component', 'Peptide_Component', 'Energy_Component',
        'Mimicry_Class', 'Quality_Weight'
    ]]

    top_candidates.to_csv('top_mimicry_candidates.csv', index=False)
    print("✓ top_mimicry_candidates.csv")
    files.download('top_mimicry_candidates.csv')

    # Component breakdown
    component_analysis = integrated_df.groupby('Mimicry_Class').agg({
        'Unified_Mimicry_Score': ['mean', 'std', 'count'],
        'TCR_Component': 'mean',
        'Peptide_Component': 'mean',
        'Energy_Component': 'mean',
        'Quality_Weight': 'mean'
    }).round(2)

    component_analysis.to_csv('mimicry_component_breakdown.csv')
    print("✓ mimicry_component_breakdown.csv")
    files.download('mimicry_component_breakdown.csv')

# ============================================================================
# FINAL SUMMARY AND INTERPRETATION
# ============================================================================
print("\n" + "="*70)
print("✅ ANALYSIS COMPLETE - FINAL SUMMARY")
print("="*70)

if integrated_df is not None:
    print(f"\n📊 UNIFIED MIMICRY ANALYSIS:")
    print(f"   • Analyzed {len(integrated_df)} peptide pairs")
    print(f"   • Mean Unified Mimicry Score: {integrated_df['Unified_Mimicry_Score'].mean():.1f}")
    print(f"   • Critical risk pairs (>70): {(integrated_df['Unified_Mimicry_Score'] > 70).sum()}")
    print(f"   • High risk pairs (50-70): {((integrated_df['Unified_Mimicry_Score'] > 50) & (integrated_df['Unified_Mimicry_Score'] <= 70)).sum()}")

if rnaseq_df is not None:
    print(f"\n🧬 TRANSCRIPTOMIC DATA:")
    print(f"   • {rnaseq_df.shape[0]} genes analyzed")
    print(f"   • {rnaseq_df.shape[1]} samples profiled")

if clinical_df is not None:
    print(f"\n🏥 CLINICAL DATA:")
    print(f"   • {len(clinical_df)} patients")
    if 'Disease_Status' in clinical_df.columns:
        print(f"   • Disease groups: {', '.join(clinical_df['Disease_Status'].unique())}")

print("\n" + "="*70)
print("INTERPRETATION GUIDE")
print("="*70)
print("""
📌 UNIFIED MIMICRY SCORE (0-100):
   • Combines TCR binding energetics + peptide similarity + binding energy
   • >70 = CRITICAL: Strong molecular mimicry, highest autoimmune risk
   • 50-70 = HIGH: Significant cross-reactivity potential
   • 30-50 = MODERATE: Some structural/functional similarity
   • <30 = LOW: Minimal mimicry concern

🔬 COMPONENTS:
   • TCR Component: How similarly TCR recognizes both peptides
   • Peptide Component: Structural/sequence similarity
   • Energy Component: Binding energy similarity
   • Quality Weight: AlphaFold confidence (higher = more reliable)

🎯 EXPERIMENTAL PRIORITIES:
   1. Test top "Critical" pairs first
   2. Validate with in vitro T-cell assays
   3. Check patient samples for cross-reactive T-cells
   4. Correlate with disease severity/progression

📁 OUTPUT FILES:
   • unified_mimicry_scores.csv - Complete integrated dataset
   • top_mimicry_candidates.csv - Top 20 for experimental validation
   • mimicry_component_breakdown.csv - Component analysis by risk class
""")

print("\n✅ Pipeline complete! Review visualizations and exported files above.")
print("="*70)

Installing packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 14.6 MB/s eta 0:00:00
✓ Complete!
INTEGRATED MULTI-OMIC MOLECULAR MIMICRY PIPELINE

PART 1: STRUCTURAL DATA INTEGRATION

📁 Upload TCR-matched predictions CSV:


Saving top_experimental_candidates (2).csv to top_experimental_candidates (2).csv
✓ Loaded 20 TCR-matched predictions

📁 Upload PDB similarity/cross-reactivity CSV:


Saving enhanced_cross_reactivity_analysis.csv to enhanced_cross_reactivity_analysis.csv
✓ Loaded 400 PDB similarity scores

PART 2: UNIFIED MOLECULAR MIMICRY SCORE CALCULATION

🔗 Merging TCR and PDB datasets...
⚠ Column name mismatch - using all data separately

PART 3: BULK RNA-SEQ DATA INTEGRATION

📁 Upload bulk RNA-seq count matrix (CSV):
   Format: Rows=genes, Columns=samples
   First column should be gene names/IDs
Do you have RNA-seq data to upload? (yes/no): no

PART 4: CLINICAL METADATA INTEGRATION

📁 Upload clinical metadata (CSV):
   Should include: Sample_ID, Disease_Status, Age, Sex, etc.
Do you have clinical data to upload? (yes/no): no

PART 5: MULTI-OMIC INTEGRATION & CORRELATION ANALYSIS

PART 6: COMPREHENSIVE MULTI-OMIC VISUALIZATIONS

PART 8: GENERATING COMPREHENSIVE REPORTS

✅ ANALYSIS COMPLETE - FINAL SUMMARY

INTERPRETATION GUIDE

📌 UNIFIED MIMICRY SCORE (0-100):
   • Combines TCR binding energetics + peptide similarity + binding energy
   • >70 = CRITICAL: Strong 